In [2]:
import pyspark
conf = pyspark.SparkConf().set('spark.driver.host','127.0.0.1')
sc = pyspark.SparkContext('local[*]', conf=conf)

spark = (pyspark.sql.SparkSession.builder 
  .master("local[*]")
  .getOrCreate())

In [3]:
!ls

Kellysnotebook.ipynb movies.txt           requests.json
LICENSE              movies_metadata.csv  user_ratings.csv
README               pablo_notebook.ipynb users.csv
movies.csv           ratings.json         users.dat
movies.dat           ratings_df.csv       users.txt


In [1]:
import pandas as pd

In [2]:
import json

ratings = []
for line in open('ratings.json', 'r'):
    ratings.append(json.loads(line))



In [3]:
ratings_dict={}
ratings_dict['user_id'] = [i['user_id'] for i in ratings]
ratings_dict['movie_id'] = [i['movie_id'] for i in ratings]
ratings_dict['rating'] = [i['rating'] for i in ratings]
ratings_dict['timestamp'] = [i['timestamp'] for i in ratings]

ratings_df = pd.DataFrame(ratings_dict)

In [4]:
ratings = pd.read_csv('ratings_df.csv')
ratings = ratings.iloc[:,1:]
ratings_df['timestamp'] = ratings_df['timestamp'].astype(int)
ratings_df.to_csv('ratings_df.csv')

In [5]:
users = pd.read_csv('users.csv')
users = users.iloc[:, 1:]
users['Zip-code']=users["Zip-code"].apply(lambda x: x[:5])
users["Zip-code"]=users["Zip-code"].astype(int)

In [6]:
user_ratings = ratings.merge(users, left_on='user_id', right_on='UserID', how='left')
user_ratings =pd.get_dummies(user_ratings, columns=['Gender'], drop_first=True)

In [7]:
user_ratings.to_csv('user_ratings')

In [8]:
movies = pd.read_csv('movies_metadata.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
import ast
movies['genres'] = movies['genres'].apply(lambda x: ast.literal_eval(x))

In [10]:
movies['id'] = movies['id'].str.extract('(\d+)')
movies['id'] = movies['id'].astype(int)

In [24]:
genres_df = user_ratings.merge(movies, left_on='movie_id', right_on='id')

In [25]:
genres_df = genres_df.set_index('movie_id')


In [26]:
def genre_list(row):
    genres = []
    for i in row:
        genres.append(i['name'])
    if len(genres) < 3:
        return genres
    else:
        return genres[:3]

genres_df['genres'] = genres_df['genres'].apply(genre_list)



In [27]:
genres_df = pd.get_dummies(genres_df['genres'].apply(pd.Series).stack()).sum(level=0)

In [30]:

genres_df.columns
genres_df = genres_df.drop(['Aniplex','Foreign', 'TV Movie',
       'BROSTA TV', 'Carousel Productions','GoHands', 
        'Music', 'Odyssey Media', 'Pulser Productions', 'Rogue State', 
        'Telescene Film Group Productions', 'Vision View Entertainment'], axis=1)

KeyError: "['Aniplex' 'Foreign' 'TV Movie' 'BROSTA TV' 'Carousel Productions'\n 'GoHands' 'Music' 'Odyssey Media' 'Pulser Productions' 'Rogue State'\n 'Telescene Film Group Productions' 'Vision View Entertainment'] not found in axis"

In [35]:
genres_df = genres_df.iloc[:, 1:].applymap(lambda x: 1 if x > 1 else 0)

In [36]:
genres_df.to_csv('genres.csv')

In [37]:
df = user_ratings.merge(genres_df, on='movie_id', how='inner')

In [ ]:
# df.fillna(0, inplace=True)

In [38]:
df.to_csv('user_ratings_genres.csv')

In [13]:
from pyspark.sql.types import (
    ArrayType,
    AtomicType,
    BinaryType,
    BooleanType,
    ByteType,
    CloudPickleSerializer,
    DataType,
    DataTypeSingleton,
    DateConverter,
    DateType,
    DatetimeConverter,
    DecimalType,
    DoubleType,
    FloatType,
    FractionalType,
    IntegerType,
    IntegralType,
    JavaClass,
    LongType,
    MapType,
    NullType,
    NumericType,
    Row,
    ShortType,
    SparkContext,
    StringType,
    StructField,
    StructType,
    TimestampType,
    UserDefinedType,
)

schema = StructType(
    [
        StructField('index', IntegerType()),
        StructField('userId', IntegerType()),
        StructField('movieId', IntegerType()),
        StructField('rating', FloatType()),
        StructField('timestamp', LongType()),
    ]
)

movie_ratings = spark.read.csv('ratings_df.csv',
                               inferSchema=False,
                               schema=schema,
                               header=True)

In [14]:
movie_ratings = movie_ratings.drop(movie_ratings.index)

In [15]:
movie_ratings = movie_ratings.drop(movie_ratings.timestamp)

In [16]:
movie_ratings.limit(5).toPandas()

,userId,movieId,rating
0,6040,858,4.0
1,6040,2384,4.0
2,6040,593,5.0
3,6040,1961,4.0
4,6040,1419,3.0


In [17]:
movie_ratings.persist()

DataFrame[userId: int, movieId: int, rating: float]

In [18]:
(trainingdata, testdata) = movie_ratings.randomSplit([0.7,0.3], seed = 10)
print("Training Dataset Count: " + str(trainingdata.count()))
print("Test Dataset Count: " + str(testdata.count()))

Training Dataset Count: 503424
Test Dataset Count: 216525


In [19]:

from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS, ALSModel

als = ALS(
    rank=10,
    maxIter=10,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
)


In [20]:
als_model = als.fit(trainingdata)
predictions = als_model.transform(testdata)

In [21]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol='rating')
evaluator.evaluate(predictions.na.drop(), {evaluator.metricName: "rmse"})

0.8779395229385331